# Visualize and Analyze Startup Data

## 1. Setup

To prepare your environment, you need to install some packages and enter credentials for the Watson services.

## 1.1 Install the necessary packages

### Install Watson Developer Cloud Package

In [13]:
!pip install watson-developer-cloud==1.5

    100% |████████████████████████████████| 225kB 3.7MB/s eta 0:00:01
Requirement not upgraded as not directly required: requests<3.0,>=2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: python_dateutil>=2.5.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
    100% |████████████████████████████████| 307kB 3.3MB/s eta 0:00:01
    100% |████████████████████████████████| 3.1MB 337kB/s eta 0:00:01
Requirement not upgraded as not directly required: pyOpenSSL>=16.2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests<3.0,>=2.0->watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/py

### Install IBM Database Server Python Package

In [14]:
!pip install ibm_db

Requirement not upgraded as not directly required: ibm_db in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


## 1.2 Import packages and libraries

Import the packages and libraries that you'll use:

In [15]:
from bs4 import BeautifulSoup
import requests
import time
from random import randint
from IPython.display import display, HTML
#import selenium
#from selenium import webdriver
import re
import pandas as pd

from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, KeywordsOptions, SemanticRolesOptions, SentimentOptions, EmotionOptions, ConceptsOptions, CategoriesOptions

import ibm_boto3
from botocore.client import Config


import json
import nltk
import csv
import ibm_db
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from io import BytesIO

## 2. Configuration

Add configurable items of the notebook below

## 2.1 Add your service credentials from IBM Natural Language Understanding service
You must create a Watson Natural Language Understanding service on IBM Cloud. Create a service for Natural Language Understanding (NLU). Insert the apikey and url values in the variables, for your NLU in the following cell. Do not change the values of the version fields.

Run the cell.

In [16]:
apikey='7S8XwKy041-qloi6TO4zhdY-EJO2TfpMlxMhUpn3Phd2'
url='https://gateway-syd.watsonplatform.net/natural-language-understanding/api'
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    iam_api_key=apikey,
    url=url
)

## 2.2 Add your service credentials for DB2

Insert the DB2 service credentials as credentials_1 in the following cell.

In [17]:

# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'username': 'dash100406',
    'password': 'kbS_B_vOi7K0',
    'database': 'BLUDB',
    'sg_service_url': 'https://sgmanager.eu-gb.bluemix.net',
    'host': 'dashdb-entry-yp-lon02-01.services.eu-gb.bluemix.net',
    'port': '50000',
    'url': 'https://undefined'
}


## 3. Scrape Startup Information

Scrapes data based on its appears on google for the following -

* How many times it has appeared on News?
* Whether it has a Wikipedia page or not?
* Whether they have Tech blogs or not?
* Whether they are active on Social Media (Twitter, Medium, etc..)?

`Note: The Company Names have been Anonymised and replaced with names of Plants. This Pattern does not wish to put down any Company, only provides a methodology to perform Analytics on any given Data from the Web.`

In [14]:
company_name_links=['https://www.goodfirms.co/artificial-intelligence/machine-learning?sort_by=Any&rate=&location=in&employees=','https://www.goodfirms.co/directory/country/list-blockchain-technology-companies/india']

In [15]:
def DisplayHtml(htmlText):
    htmlTxt = HTML(htmlText)
    # or chart = charts.plot(...)
    display(htmlTxt)


def scrape_top_companies(link):
    time.sleep(randint(0, 2))  # relax and don't let google be angry
    #r = requests.get("http://www.google.co.uk/search?q="+s+"&tbm=nws")
    r = requests.get('https://www.goodfirms.co/artificial-intelligence/machine-learning?sort_by=Any&rate=&location=in&employees=')
    #r = requests.get("https://www.google.co.in/search?q="+s+"&oq="+s+"&aqs=chrome..69i57.14096j0j9&sourceid=chrome&ie=UTF-8")
    print(r.status_code)  # Print the status code
    content = r.text
    #DisplayHtml(content)
    company_names = []
    company_desc=[]
    soup = BeautifulSoup(content, "html.parser")
    #st_divs = soup.findAll("div", {"class": "st"})
    #st_divs = soup.findAll("a", {"class": "l _PMs"})

    for item in soup.find_all('h3'):
        for post in item.find_all('a'):
            company_names.append(post.text)
    '''<div class="company-info-title pull-left">
<h3 class="font20"><span itemprop="position">1.</span> <a class="font20 c_name_head weight700 detail_page" href="/companies/view/7622/arm" target="_blank" title="Arm"><span itemprop="name">Arm</span></a></h3>
</div>'''
    for item in soup.find_all('p',{'class':'nofollow padding-top-10 no-margin'}):
        company_desc.append(item.text)
    '''<p class="nofollow padding-top-10 no-margin">
Arm technology is at the heart of a computing and connectivity revolution that is transforming the way people live and businesses operate. Our advanced, energy-efficient processor designs are enabling the intelligence in 100 billion silicon chips and securely powering products from the sensor to the smartphone to the supercomputer.
</p>'''
    #for st_div in st_divs:
        #news_summaries.append(st_div.text)
        
    company_links=[]
    for data in soup.find_all('div',{'class':'visit-website-div nofollow'}):
        for a in data.find_all('a'):
            company_links.append(a.get('href'))
    
    print(len(company_names),len(company_desc))
    return company_names,company_desc,company_links


In [16]:
company_list_final=[]
dummy_names = ['Wildflower', 'Thistle', 'Flower', 'Herb', 'Mushroom', 'Weed', 'Fern', 'Cattail', 'Reed', 'Bamboo', 'Ivy', 'Shrub', 'Moss', 'Grass', 'Palm tree', 'Bush', 'Corn', 'Tree', 'Tulip', 'Daffodil', 'Poppy', 'Sunflower', 'Bluebell', 'Rose', 'Snowdrop', 'Cherry blossom', 'Orchid', 'Iris', 'Peony', 'Chrysanthemum', 'Geranium', 'Lily', 'Lotus', 'Water lily', 'Dandelion']
#for c in company_name_links:
link='https://www.goodfirms.co/artificial-intelligence/machine-learning?sort_by=Any&rate=&location=in&employees='
company_names,company_desc,company_links=scrape_top_companies(link)
for i in range(len(company_names)):
    temp=dict()
    temp[dummy_names[i]]=[company_desc[i],company_links[i]]
    company_list_final.append(temp)
company_list_final

200
35 35


[{'Wildflower': ['\nOur services are designed to take your business to the next level, reducing the gap between you and the heights of success you wish to achieve.\nEmpowered by 12+ years of experience in the field of IT, CIS has the right expertise to help you with all your technology needs. We have consistently delivered optimal solutions for our wide-ranging global client base. Leverage our rich experience for your unique IT requirements.\nCIS has established ... Read more ',
   'https://www.cisin.com']},
 {'Thistle': ['\nVOLANSYS is a Silicon Valley based product realization and digital transformation company offering embedded systems, digital networking, internet of things, cloud and mobility solutions from concept to roll-out. Since 2008, VOLANSYS has been enabling worldwide customers including fortune 500 companies as well as growth-stage start-ups to bring ideas to life at faster time to market. With 50+ products implemented, 150+ customers, 350+ professional ... Read more ',
 

In [17]:
def scrape_news_summaries_google(s):
    time.sleep(randint(0, 2))  # relax and don't let google be angry
    #r = requests.get("http://www.google.co.uk/search?q="+s+"&tbm=nws")
    r = requests.get("https://www.google.co.in/search?q="+s+"&oq="+s+"&aqs=chrome..69i57.14096j0j9&sourceid=chrome&ie=UTF-8")
    print(r.status_code)  # Print the status code
    content = r.text
    #DisplayHtml(content)
    news_items=[]
    
    soup = BeautifulSoup(content, "html.parser")
    #st_divs = soup.findAll("div", {"class": "st"})
    #st_divs = soup.findAll("a", {"class": "l _PMs"})  
    
    for item in soup.find_all(class_='g'):
        news_dict=dict()
        for link in item.find_all('a'):
            news_dict['news_link']= link.get('href')
        for summary in item.find_all('span',class_="st"): 
            news_dict['summary']=summary.text
        news_items.append(news_dict)
    return news_items



In [18]:
final_rows=[]
val=dict()
for c in company_list_final:
    for key,value in c.items():
        s='"'+key+'"'+'company economic times'
        inner_dict=dict()
        temp=[]
        temp= temp + scrape_news_summaries_google(s)
        inner_dict['Description']=value[0]
        inner_dict['Company_Link']=value[1]
        inner_dict['News_Items']=temp
        val[key]=inner_dict
        final_rows.append(val)
        val=dict()

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [19]:
final_rows[0]

{'Wildflower': {'Company_Link': 'https://www.cisin.com',
  'Description': '\nOur services are designed to take your business to the next level, reducing the gap between you and the heights of success you wish to achieve.\nEmpowered by 12+ years of experience in the field of IT, CIS has the right expertise to help you with all your technology needs. We have consistently delivered optimal solutions for our wide-ranging global client base. Leverage our rich experience for your unique IT requirements.\nCIS has established ... Read more ',
  'News_Items': [{'news_link': '/url?q=https://economictimes.indiatimes.com/company/wildflower-club-%26-resort-private-limited/U55209RJ2018PTC061584&sa=U&ved=0ahUKEwiN89S0sabgAhUCVrwKHUhjAv8QFggUMAA&usg=AOvVaw2zX7vGtqs77WV-6dRwXLvq',
    'summary': "Wildflower Club & Resort Private Limited is an Indian Non-Government Company\n. It's a private company and is classified as 'company limited by shares'."},
   {'news_link': '/url?q=https://economictimes.indiat

In [20]:
def split_sentences(text):
    """ Split text into sentences.
    """
    sentence_delimiters = re.compile(u'[\\[\\]\n.!?]')
    sentences = sentence_delimiters.split(text)
    return sentences

def split_into_tokens(text):
    """ Split text into tokens.
    """
    tokens = nltk.word_tokenize(text)
    return tokens

def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read()
    return text.decode('utf-8')

def POS_tagging(text):
    """ Generate Part of speech tagging of the text.
    """
    POSofText = nltk.tag.pos_tag(text)
    return POSofText

def resolve_coreference(text, config):
    """ Resolve coreferences in the text for Nouns that are Subjects in a sentence
    """
    sentenceList = split_sentences(text)
    referenceSubject = ''
    sentenceText = ''
    configjson = json.loads(config)
    
    for sentences in sentenceList:    
        tokens = split_into_tokens(sentences)   
        postags = POS_tagging(tokens)
        sentencetags = chunk_sentence(postags)
        subjects = find_subject(sentencetags)
        for rules in configjson['configuration']['coreference']['rules']:
            if (rules['type'] == 'chunking'):
                for tags in rules['chunk']:
                    chunktags = chunk_tagging(tags['tag'],tags['pattern'],postags)
                    if (len(chunktags)>0):
                        for words in chunktags:
                            if tags['tag'] == 'PRP':
                                if subjects == '':
                                    sentenceText = sentenceText+sentences.replace(words,referenceSubject)+'. '
                            elif tags['tag'] == 'NAME':
                                if words == subjects:
                                    referenceSubject = words
                                    sentenceText = sentenceText+sentences+'. '
                    
    return sentenceText

def chunk_sentence(text):
    """ Tag the sentence using chunking.
    """
    grammar = """
      NP: {<DT|JJ|PRP|NN.*>+} # Chunk sequences of DT,JJ,NN
          #}<VB*|DT|JJ|RB|PRP><NN.*>+{  # Chink sequences of VB,DT,JJ,NN       
      PP: {<IN><NP>}               # Chunk prepositions followed by NP
      V: {<V.*>}                   # Verb      
      VP: {<VB*><NP|PP|CLAUSE>+}  # Chunk verbs and their arguments
      CLAUSE: {<NP><VP>}           # Chunk NP, VP
      """  
    parsed_cp = nltk.RegexpParser(grammar,loop=2)
    pos_cp = parsed_cp.parse(text)
    return pos_cp

def find_subject(t):
    for s in t.subtrees(lambda t: t.label() == 'NP'):
        return find_attrs(s,'NP')
    
def find_attrs(subtree,phrase):
    attrs = ''
    if phrase == 'NP':
        for nodes in subtree:
            if nodes[1] in ['DT','PRP$','POS','JJ','CD','ADJP','QP','NP','NNP']:
                attrs = attrs+' '+nodes[0]
    return attrs   

def chunk_tagging(tag,chunk,text):
    """ Tag the text using chunking.
    """
    parsed_cp = nltk.RegexpParser(chunk)
    pos_cp = parsed_cp.parse(text)
    chunk_list=[]
    for root in pos_cp:
        if isinstance(root, nltk.tree.Tree):               
            if root.label() == tag:
                chunk_word = ''
                for child_root in root:
                    chunk_word = chunk_word +' '+ child_root[0]
                chunk_list.append(chunk_word)
    return chunk_list

def analyze_using_NLU(analysistext):
    """ Extract results from Watson Natural Language Understanding for each news item
    """
    res=dict()
    response = natural_language_understanding.analyze( 
        text=analysistext,
        features=Features(
                          sentiment=SentimentOptions(),
                          entities=EntitiesOptions(), 
                          keywords=KeywordsOptions(),
                          emotion=EmotionOptions(),
                          concepts=ConceptsOptions(),
                          categories=CategoriesOptions(),
                          semantic_roles=SemanticRolesOptions()))
    res['results']=response
    return res



In [21]:
def hasET(company_name):
    cnbcVal=0
    cnbcLinks=[]
    ET_link=[]
    s='"'+company_name+'"'+' economic times'
    res= scrape_news_summaries_google(s)
    return res

def hasTwitter(company_name):
    cnbcVal=0
    cnbcLinks=[]
    ET_link=[]
    s='"'+company_name+'"'+' twitter'
    res= scrape_news_summaries_google(s)
    return res


def getTechAreaNews(article_text):
    concept=''
    relevance=''
    if article_text != '':
        NLUres=analyze_using_NLU(article_text)
        
        if len(NLUres['results']['concepts']) != 0:
            concept=NLUres['results']['concepts'][0]['text']
            relevance=NLUres['results']['concepts'][0]['relevance']
        if len(NLUres['results']['sentiment']) != 0: 
            sentiment=NLUres['results']['sentiment']['document']['label']
    return concept,relevance,sentiment

def getTechArea(article_text):
    concept=''
    relevance=''
    sentiment=''
    if article_text != '':
        NLUres=analyze_using_NLU(article_text)
        if len(NLUres['results']['concepts']) != 0:
            concept=NLUres['results']['concepts'][0]['text']
            relevance=NLUres['results']['concepts'][0]['relevance']
    return concept,relevance

def hasWiki(s):
    wikiVal=0
    wikiLinks=[]
    s=s.replace(' ','+')
    link='https://en.wikipedia.org/w/index.php?search='+s+'&title=Special%3ASearch&go=Go'
    r = requests.get(link)
    print(r.status_code)
    content = r.text
    return content
    

## 3.1 Collect Wiki

Collects info on how many Companies have an existing Wikipedia page

In [22]:
wikiList=[]
for f in final_rows:
    for name, info in f.items():
        wiki=dict()
        wiki['Company_Name']=name
        wiki['Wiki_Concept'],wiki['Wiki_Confidence']=getTechArea(hasWiki(name))
        wikiList.append(wiki)
wikiList

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


[{'Company_Name': 'Wildflower',
  'Wiki_Concept': 'Native plant',
  'Wiki_Confidence': 0.917512},
 {'Company_Name': 'Thistle',
  'Wiki_Concept': 'Thistle',
  'Wiki_Confidence': 0.965009},
 {'Company_Name': 'Flower',
  'Wiki_Concept': 'Flower',
  'Wiki_Confidence': 0.952994},
 {'Company_Name': 'Herb', 'Wiki_Concept': 'Herb', 'Wiki_Confidence': 0.944664},
 {'Company_Name': 'Mushroom',
  'Wiki_Concept': 'Agaricales',
  'Wiki_Confidence': 0.978327},
 {'Company_Name': 'Weed', 'Wiki_Concept': 'Weed', 'Wiki_Confidence': 0.984223},
 {'Company_Name': 'Fern', 'Wiki_Concept': 'Fern', 'Wiki_Confidence': 0.946545},
 {'Company_Name': 'Cattail',
  'Wiki_Concept': 'Typha',
  'Wiki_Confidence': 0.973402},
 {'Company_Name': 'Reed', 'Wiki_Concept': '', 'Wiki_Confidence': ''},
 {'Company_Name': 'Bamboo',
  'Wiki_Concept': 'Bamboo',
  'Wiki_Confidence': 0.896661},
 {'Company_Name': 'Ivy', 'Wiki_Concept': 'Ivy', 'Wiki_Confidence': 0.964382},
 {'Company_Name': 'Shrub',
  'Wiki_Concept': 'Shrub',
  'Wiki_Conf

In [23]:
keys = wikiList[0].keys()
with open('Wiki_Blockchain.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(wikiList)

## 3.2 Collect ET

Collects info through google links on how many hits are articles are from ET for a particular company

In [24]:
ET=[]
for f in final_rows:
    for name, info in f.items():
        temp=dict()
        news=hasET(name)
        for n in news:
            flag=0
            if 'summary' in n:
                summary=n['summary']
                flag=1
            #print(summary)
            link=n['news_link']
            temp=dict()
            #print('economictimes' in link)
            if 'economictimes' in link and flag:
                    temp['Company_Name']=name
                    temp['News_Link']=link
                    temp['News_Concept'],temp['News_Relevance'],temp['News_Sentiment']=getTechAreaNews(summary)
                    ET.append(temp)
ET

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


[{'Company_Name': 'Wildflower',
  'News_Concept': 'Broadsheet',
  'News_Link': '/url?q=https://economictimes.indiatimes.com/topic/wildflower-hall/news/2/2&sa=U&ved=0ahUKEwizoeHOsqbgAhW3UhUIHWWgDXAQFggUMAA&usg=AOvVaw0yeKSzv7bToDyfPbv3Q1M-',
  'News_Relevance': 0.888655,
  'News_Sentiment': 'neutral'},
 {'Company_Name': 'Wildflower',
  'News_Concept': 'Broadsheet',
  'News_Link': '/url?q=https://economictimes.indiatimes.com/topic/wildflower-hall&sa=U&ved=0ahUKEwizoeHOsqbgAhW3UhUIHWWgDXAQFggWMAE&usg=AOvVaw3IK0rvv7LNDkN7qpVOxOOH',
  'News_Relevance': 0.909961,
  'News_Sentiment': 'negative'},
 {'Company_Name': 'Wildflower',
  'News_Concept': 'Lord Kitchener',
  'News_Link': '/url?q=http://webcache.googleusercontent.com/search%3Fq%3Dcache:M5KLI4cZ4gIJ:https://economictimes.indiatimes.com/wildflower-hall-mashobra/slideshow/3019771.cms%252B%2522Wildflower%2522%2Beconomic%2Btimes%26hl%3Den%26ct%3Dclnk&sa=U&ved=0ahUKEwizoeHOsqbgAhW3UhUIHWWgDXAQIAgbMAI&usg=AOvVaw0oFcxIQEZYkQ7rQMOwRXrQ',
  'News_

In [25]:
keys = ET[0].keys()
with open('ET_final_Blockchain.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(ET)

## 3.3 Collect Tech Area

Suggests the major tech area of a company

In [26]:
tech_area=[]
for f in final_rows:
    for name,info in f.items():
        temp=dict()
        temp["Company_Name"]=name
        print(info)
        temp["Technology"],temp["Technology_Relevance"]=getTechArea(info['Description'])
        tech_area.append(temp)

{'Description': '\nOur services are designed to take your business to the next level, reducing the gap between you and the heights of success you wish to achieve.\nEmpowered by 12+ years of experience in the field of IT, CIS has the right expertise to help you with all your technology needs. We have consistently delivered optimal solutions for our wide-ranging global client base. Leverage our rich experience for your unique IT requirements.\nCIS has established ... Read more ', 'Company_Link': 'https://www.cisin.com', 'News_Items': [{'summary': "Wildflower Club & Resort Private Limited is an Indian Non-Government Company\n. It's a private company and is classified as 'company limited by shares'.", 'news_link': '/url?q=https://economictimes.indiatimes.com/company/wildflower-club-%26-resort-private-limited/U55209RJ2018PTC061584&sa=U&ved=0ahUKEwiN89S0sabgAhUCVrwKHUhjAv8QFggUMAA&usg=AOvVaw2zX7vGtqs77WV-6dRwXLvq'}, {'summary': '15 Sep 2018 ... Get the latest Wildflower Estate And Resorts Pr

In [27]:
tech_area

[{'Company_Name': 'Wildflower', 'Technology': '', 'Technology_Relevance': ''},
 {'Company_Name': 'Thistle',
  'Technology': 'Fortune 500',
  'Technology_Relevance': 0.979701},
 {'Company_Name': 'Flower',
  'Technology': 'Suffering',
  'Technology_Relevance': 0.865846},
 {'Company_Name': 'Herb',
  'Technology': 'New York City',
  'Technology_Relevance': 0.975634},
 {'Company_Name': 'Mushroom',
  'Technology': 'Outsourcing',
  'Technology_Relevance': 0.973445},
 {'Company_Name': 'Weed',
  'Technology': 'Design',
  'Technology_Relevance': 0.936521},
 {'Company_Name': 'Fern',
  'Technology': 'Developmental biology',
  'Technology_Relevance': 0.902641},
 {'Company_Name': 'Cattail',
  'Technology': 'Data analysis',
  'Technology_Relevance': 0.978711},
 {'Company_Name': 'Reed',
  'Technology': 'Silicon Valley',
  'Technology_Relevance': 0.948182},
 {'Company_Name': 'Bamboo',
  'Technology': 'Fortune 500',
  'Technology_Relevance': 0.979701},
 {'Company_Name': 'Ivy',
  'Technology': 'Capabilit

In [28]:
keys = tech_area[0].keys()
with open('Tech_Area_Final_Blockchain.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(tech_area)

## 3.4 Collect Twitter

Collects info on how many Tweets appear on Google Search of a Company

In [29]:
Twitter=[]
for f in final_rows:
    for name, info in f.items():
        temp=dict()
        news=hasTwitter(name)
        for n in news:
            flag=0
            if 'summary' in n:
                summary=n['summary']
                flag=1
            #print(summary)
            link=n['news_link']
            temp=dict()
            #print('economictimes' in link)
            if 'twitter.com/'+name.lower() in link and flag:
                    temp['name']=name
                    temp['link']=link
                    temp['concept'],temp['Confidence'],temp['Sentiment']=getTechAreaNews(summary)
                    Twitter.append(temp)
Twitter

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
503
503
503
503
503
503
503
503
503
503
503
503
503
503
503
503


[{'Confidence': 0.909247,
  'Sentiment': 'positive',
  'concept': 'Bag-In-Box',
  'link': '/search?ie=UTF-8&q=related:https://twitter.com/wildflowercases%3Flang%3Den+%22Wildflower%22+twitter&tbo=1&sa=X&ved=0ahUKEwjGrM2KtKbgAhWJNd8KHdNkAj8QHwgeMAE',
  'name': 'Wildflower'},
 {'Confidence': 0.954089,
  'Sentiment': 'negative',
  'concept': 'World War II',
  'link': '/url?q=http://webcache.googleusercontent.com/search%3Fq%3Dcache:KyMJiQaJcFYJ:https://twitter.com/wildflower_hour%3Flang%253Den%252B%2522Wildflower%2522%2Btwitter%26hl%3Den%26ct%3Dclnk&sa=U&ved=0ahUKEwjGrM2KtKbgAhWJNd8KHdNkAj8QIAgoMAM&usg=AOvVaw3rNKXrrgJeVSipVad-SuEP',
  'name': 'Wildflower'},
 {'Confidence': '',
  'Sentiment': 'neutral',
  'concept': '',
  'link': '/url?q=http://webcache.googleusercontent.com/search%3Fq%3Dcache:xi8VcLyp6QgJ:https://twitter.com/wildflowerse15%3Flang%253Den%252B%2522Wildflower%2522%2Btwitter%26hl%3Den%26ct%3Dclnk&sa=U&ved=0ahUKEwjGrM2KtKbgAhWJNd8KHdNkAj8QIAgtMAQ&usg=AOvVaw0G-TtCiP1UAeB4s0xEUf3J

In [30]:
keys = Twitter[0].keys()
with open('Twitter_Blockchain.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(Twitter)

## 4. Combine the Results and Save to SPSS

In [31]:
file = open("Tech_Area_Final_Blockchain.csv", "r")
Tech_area = pd.read_csv(file, delimiter=',')
file = open("ET_final_Blockchain.csv", "r")
ET = pd.read_csv(file, delimiter=',')
file = open("Wiki_Blockchain.csv", "r")
Wiki = pd.read_csv(file, delimiter=',')
file = open("Twitter_Blockchain.csv", "r")
Twitter = pd.read_csv(file, delimiter=',')

In [32]:
Twitter

,Sentiment,link,concept,name,Confidence
0,positive,/search?ie=UTF-8&q=related:https://twitter.com...,Bag-In-Box,Wildflower,0.909247
1,negative,/url?q=http://webcache.googleusercontent.com/s...,World War II,Wildflower,0.954089
2,neutral,/url?q=http://webcache.googleusercontent.com/s...,NaN,Wildflower,NaN
3,negative,/search?ie=UTF-8&q=related:https://twitter.com...,NaN,Wildflower,NaN
4,neutral,/search?ie=UTF-8&q=related:https://twitter.com...,Australia,Wildflower,0.911360
5,neutral,/url?q=http://webcache.googleusercontent.com/s...,Wind,Wildflower,0.886784
6,positive,/search?ie=UTF-8&q=related:https://twitter.com...,NaN,Wildflower,NaN
7,negative,/search?ie=UTF-8&q=related:https://twitter.com...,Ireland,Thistle,0.968453
8,positive,/search?ie=UTF-8&q=related:https://twitter.com...,Customer service,Thistle,0.945319
9,positive,/search?ie=UTF-8&q=related:https://twitter.com...,NaN,Thistle,NaN


In [33]:
Twitter=Twitter.rename(index=str,columns={'name':'Company_Name','link':'Twitter_news_link','concept':'Twitter_Topic','confidence':'Twitter_Relevance','Sentiment':'Twitter_Sentiment'})

In [80]:
Twitter

,Twitter_Topic,Confidence,Company_Name,Twitter_Sentiment,Twitter_news_link
0,Mobile computing,0.904400,ArStudioz,neutral,/url?q=http://webcache.googleusercontent.com/s...
1,Mobile computing,0.904400,ArStudioz,positive,/url?q=http://webcache.googleusercontent.com/s...
2,Mobile computing,0.904400,ArStudioz,positive,/url?q=http://webcache.googleusercontent.com/s...
3,Michigan,0.842018,Datamatics,positive,/url?q=http://webcache.googleusercontent.com/s...
4,NaN,NaN,Datamatics,positive,/url?q=http://webcache.googleusercontent.com/s...
5,India,0.934862,BinaryBoot,neutral,/url?q=http://webcache.googleusercontent.com/s...
6,Chemistry,0.829241,GoodWorkLabs,positive,/search?ie=UTF-8&q=related:https://twitter.com...
7,Chemistry,0.829241,GoodWorkLabs,positive,/url?q=http://webcache.googleusercontent.com/s...
8,NaN,NaN,Accubits,positive,/url?q=http://webcache.googleusercontent.com/s...
9,Recruitment,0.855158,Accubits,positive,/url?q=http://webcache.googleusercontent.com/s...


In [34]:
a = ET.append(Wiki)

In [35]:
b=a.append(Twitter)

In [36]:
b

,Company_Name,Confidence,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence
0,Wildflower,NaN,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.888655,neutral,NaN,NaN,NaN,NaN,NaN
1,Wildflower,NaN,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,negative,NaN,NaN,NaN,NaN,NaN
2,Wildflower,NaN,Lord Kitchener,/url?q=http://webcache.googleusercontent.com/s...,0.911360,positive,NaN,NaN,NaN,NaN,NaN
3,Wildflower,NaN,Corporation,/url?q=https://economictimes.indiatimes.com/co...,0.972000,positive,NaN,NaN,NaN,NaN,NaN
4,Wildflower,NaN,Private company limited by shares,/url?q=https://economictimes.indiatimes.com/co...,0.877024,neutral,NaN,NaN,NaN,NaN,NaN
5,Wildflower,NaN,Corporation,/url?q=http://webcache.googleusercontent.com/s...,0.947700,positive,NaN,NaN,NaN,NaN,NaN
6,Thistle,NaN,Broadsheet,/url?q=http://webcache.googleusercontent.com/s...,0.909961,negative,NaN,NaN,NaN,NaN,NaN
7,Thistle,NaN,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,negative,NaN,NaN,NaN,NaN,NaN
8,Thistle,NaN,Broadsheet,/url?q=https://economictimes.indiatimes.com/co...,0.941908,neutral,NaN,NaN,NaN,NaN,NaN
9,Thistle,NaN,Broadsheet,/url?q=http://webcache.googleusercontent.com/s...,0.909961,negative,NaN,NaN,NaN,NaN,NaN


In [58]:
compiled_rows=pd.merge(b,Tech_area, on="Company_Name")

In [59]:
compiled_rows

,Company_Name,Confidence,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,Wildflower,NaN,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.888655,neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wildflower,NaN,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wildflower,NaN,Lord Kitchener,/url?q=http://webcache.googleusercontent.com/s...,0.911360,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wildflower,NaN,Corporation,/url?q=https://economictimes.indiatimes.com/co...,0.972000,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wildflower,NaN,Private company limited by shares,/url?q=https://economictimes.indiatimes.com/co...,0.877024,neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Wildflower,NaN,Corporation,/url?q=http://webcache.googleusercontent.com/s...,0.947700,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Wildflower,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Native plant,0.917512,NaN,NaN
7,Wildflower,0.909247,NaN,NaN,NaN,NaN,positive,Bag-In-Box,/search?ie=UTF-8&q=related:https://twitter.com...,NaN,NaN,NaN,NaN
8,Wildflower,0.954089,NaN,NaN,NaN,NaN,negative,World War II,/url?q=http://webcache.googleusercontent.com/s...,NaN,NaN,NaN,NaN
9,Wildflower,NaN,NaN,NaN,NaN,NaN,neutral,NaN,/url?q=http://webcache.googleusercontent.com/s...,NaN,NaN,NaN,NaN


In [60]:
import numpy as np
compiled_rows = compiled_rows.replace(np.nan, '', regex=True)

In [61]:
compiled_rows

,Company_Name,Confidence,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,Wildflower,,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.888655,neutral,,,,,,,
1,Wildflower,,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,negative,,,,,,,
2,Wildflower,,Lord Kitchener,/url?q=http://webcache.googleusercontent.com/s...,0.91136,positive,,,,,,,
3,Wildflower,,Corporation,/url?q=https://economictimes.indiatimes.com/co...,0.972,positive,,,,,,,
4,Wildflower,,Private company limited by shares,/url?q=https://economictimes.indiatimes.com/co...,0.877024,neutral,,,,,,,
5,Wildflower,,Corporation,/url?q=http://webcache.googleusercontent.com/s...,0.9477,positive,,,,,,,
6,Wildflower,,,,,,,,,Native plant,0.917512,,
7,Wildflower,0.909247,,,,,positive,Bag-In-Box,/search?ie=UTF-8&q=related:https://twitter.com...,,,,
8,Wildflower,0.954089,,,,,negative,World War II,/url?q=http://webcache.googleusercontent.com/s...,,,,
9,Wildflower,,,,,,neutral,,/url?q=http://webcache.googleusercontent.com/s...,,,,


In [44]:
compiled_rows['Wiki_Confidence'].fillna(0.0, inplace=True)
compiled_rows

,Company_Name,Confidence,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,Wildflower,NaN,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.888655,neutral,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
1,Wildflower,NaN,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,negative,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
2,Wildflower,NaN,Lord Kitchener,/url?q=http://webcache.googleusercontent.com/s...,0.911360,positive,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
3,Wildflower,NaN,Corporation,/url?q=https://economictimes.indiatimes.com/co...,0.972000,positive,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
4,Wildflower,NaN,Private company limited by shares,/url?q=https://economictimes.indiatimes.com/co...,0.877024,neutral,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
5,Wildflower,NaN,Corporation,/url?q=http://webcache.googleusercontent.com/s...,0.947700,positive,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
6,Wildflower,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Native plant,0.917512,NaN,NaN
7,Wildflower,0.909247,NaN,NaN,NaN,NaN,positive,Bag-In-Box,/search?ie=UTF-8&q=related:https://twitter.com...,NaN,0.000000,NaN,NaN
8,Wildflower,0.954089,NaN,NaN,NaN,NaN,negative,World War II,/url?q=http://webcache.googleusercontent.com/s...,NaN,0.000000,NaN,NaN
9,Wildflower,NaN,NaN,NaN,NaN,NaN,neutral,NaN,/url?q=http://webcache.googleusercontent.com/s...,NaN,0.000000,NaN,NaN


In [45]:
compiled_rows['Confidence'].fillna(0.0, inplace=True)
compiled_rows

,Company_Name,Confidence,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,Wildflower,0.000000,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.888655,neutral,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
1,Wildflower,0.000000,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,negative,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
2,Wildflower,0.000000,Lord Kitchener,/url?q=http://webcache.googleusercontent.com/s...,0.911360,positive,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
3,Wildflower,0.000000,Corporation,/url?q=https://economictimes.indiatimes.com/co...,0.972000,positive,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
4,Wildflower,0.000000,Private company limited by shares,/url?q=https://economictimes.indiatimes.com/co...,0.877024,neutral,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
5,Wildflower,0.000000,Corporation,/url?q=http://webcache.googleusercontent.com/s...,0.947700,positive,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
6,Wildflower,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Native plant,0.917512,NaN,NaN
7,Wildflower,0.909247,NaN,NaN,NaN,NaN,positive,Bag-In-Box,/search?ie=UTF-8&q=related:https://twitter.com...,NaN,0.000000,NaN,NaN
8,Wildflower,0.954089,NaN,NaN,NaN,NaN,negative,World War II,/url?q=http://webcache.googleusercontent.com/s...,NaN,0.000000,NaN,NaN
9,Wildflower,0.000000,NaN,NaN,NaN,NaN,neutral,NaN,/url?q=http://webcache.googleusercontent.com/s...,NaN,0.000000,NaN,NaN


In [46]:
compiled_rows['News_Relevance'].fillna(0.0, inplace=True)
compiled_rows

,Company_Name,Confidence,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,Wildflower,0.000000,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.888655,neutral,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
1,Wildflower,0.000000,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,negative,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
2,Wildflower,0.000000,Lord Kitchener,/url?q=http://webcache.googleusercontent.com/s...,0.911360,positive,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
3,Wildflower,0.000000,Corporation,/url?q=https://economictimes.indiatimes.com/co...,0.972000,positive,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
4,Wildflower,0.000000,Private company limited by shares,/url?q=https://economictimes.indiatimes.com/co...,0.877024,neutral,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
5,Wildflower,0.000000,Corporation,/url?q=http://webcache.googleusercontent.com/s...,0.947700,positive,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
6,Wildflower,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,Native plant,0.917512,NaN,NaN
7,Wildflower,0.909247,NaN,NaN,0.000000,NaN,positive,Bag-In-Box,/search?ie=UTF-8&q=related:https://twitter.com...,NaN,0.000000,NaN,NaN
8,Wildflower,0.954089,NaN,NaN,0.000000,NaN,negative,World War II,/url?q=http://webcache.googleusercontent.com/s...,NaN,0.000000,NaN,NaN
9,Wildflower,0.000000,NaN,NaN,0.000000,NaN,neutral,NaN,/url?q=http://webcache.googleusercontent.com/s...,NaN,0.000000,NaN,NaN


In [50]:
import numpy as np
compiled_rows = compiled_rows.replace(np.nan, '', regex=True)

### Store and Add table in DB2 Warehouse 

In [18]:
dsn_driver = "IBM DB2 ODBC DRIVER"
dsn_database = credentials_1['database'] 
dsn_hostname = credentials_1['host']
dsn_port = 50000               
dsn_uid = credentials_1['username']      
dsn_pwd = credentials_1['password']

dsn = (
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE="+str(dsn_database)+";"
    "HOSTNAME="+str(dsn_hostname)+";"
    "PORT="+str(dsn_port)+";"
    "PROTOCOL=TCPIP;"
    "UID="+str(dsn_uid)+";"
    "PWD="+str(dsn_pwd)+";").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)

conn = ibm_db.connect(dsn, "", "")

In [64]:
create_statement=pd.io.sql.get_schema(compiled_rows.reset_index(), 'DATA_FOR_SPSS')
create_statement=create_statement.replace('TEXT', 'VARCHAR(500)')
ibm_db.exec_immediate(conn, create_statement)

In [65]:
tuple_of_tuples = tuple([tuple(x) for x in compiled_rows.values])
i=1
for x in compiled_rows.values:
    #print((tuple(x)))
    vals= (i,) + tuple(x)
    #print(vals)
    #print(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[11],x[12])
    sql = "INSERT INTO DASH100406.DATA_FOR_SPSS VALUES"+ str(vals)
    i=i+1
    ins_sql=ibm_db.prepare(conn, sql)
    ibm_db.execute(ins_sql)


In [23]:
create_statement= 'CREATE TABLE "DATA_FOR_COGNOS" (\n"index" SMALLINT,\n "Company_Name" VARCHAR,\n  "Confidence" DECFLOAT,\n  "News_Concept" VARCHAR,\n  "News_Link" VARCHAR, \n  "News_Relevance" DECFLOAT,\n "Overall_Sentiment" VARCHAR ,\n  "Twitter_Topic" VARCHAR,\n  "Twitter_news_link" VARCHAR,\n  "Wiki_Concept" VARCHAR,\n "Wiki_Confidence" VARCHAR,\n  "Technology" VARCHAR,\n  "Technology_Relevance" DECFLOAT,\n "Company_News_Sentiments" SMALLINT\n)'
create_statement=create_statement.replace('VARCHAR', 'VARCHAR(500)')
ibm_db.exec_immediate(conn, create_statement)